In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(max_tokens=1024, temperature=0.1)

In [30]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.split(",")
        return list(map(str.strip, items))
    

p = CommaOutputParser()
p.parse(" Hello, How ,are, you")

['Hello', 'How', 'are', 'you']

In [39]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items}. in lower case. Do Not Reply with anything else"),
    ("human", "{question}")
])


In [43]:
chain = template | chat | CommaOutputParser()
chain.invoke({
    "max_items": 5,
    "question": "포켓몬이 뭐야?"
})

['피카츄', '파이리', '꼬부기', '이상해씨']